In [1]:
import numpy as np
import pygame as pg
import math
import time

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
WIDTH = 600
HEIGHT = 600
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

In [7]:
class Turret:
    def __init__(self, screen, width, height):
        self.pos_x = width/2
        self.pos_y = height/2
        self.position = np.array([self.pos_x, self.pos_y], dtype=np.float32)
        self.screen = screen
        self.angle = 0
        self.radius = 15
        
        self.shoot_interval = 2 # shooting interval of the turret
        self.center = (int(self.pos_x), int(self.pos_y))

        self.rect = pg.draw.circle(screen, GREEN, self.center, self.radius)

        self.bullet_pos = (self.center[0]+ self.radius * math.cos(math.radians(angle)), self.center[1]+ self.radius * math.sin(math.radians(angle)))

        self.bullets = []

        
    def rotate_turret(self, end_point=None):
        """
        the end_point here gives a position coordinate (x, y)
        this needs to be used for the turret direction
        """
        
        magnitude = 30
        angle = math.radians(self.angle)
        # angle = self.angle
        dir_vec_x = magnitude * math.cos(angle)
        dir_vec_y = magnitude * math.sin(angle)

        direction_end = (self.center[0]+ dir_vec_x, self.center[1] + dir_vec_y)

        self.bullet_pos = (self.center[0]+ self.radius * math.cos(angle), self.center[1]+ self.radius * math.sin(angle))
        self.rect = pg.draw.circle(screen, GREEN, self.center, self.radius)
        # self.bullet_pos = direction_end / 2

        return direction_end

    def shoot(self):
        bullet = Bullet(*self.bullet_pos, self.angle, self.screen)
        self.bullets.append(bullet)
        # if shooting interval is 2 then shoot at the current angle
    def get_bullets(self):
        return self.bullets
        
    def destroy_bullet(self, bullet):
        self.bullets.remove(bullet)
        

In [8]:
class Bullet:
    def __init__(self, x, y, angle, screen):
        self.width = 2
        self.height = 5
        self.angle = angle
        self.speed = 5

        self.pos_x = x
        self.pos_y = y
        self.pos = np.array([self.pos_x, self.pos_y], dtype=np.float32)
        self.center = (int(self.pos_x), int(self.pos_y))
        self.radius = 10
        
        # this rect object is for drawing
        self.rect = pg.draw.circle(screen, RED, self.center, self.radius)
        
    def move(self):
        angle = math.radians(self.angle)
        # self.pos_x += self.speed * math.cos(angle)
        # self.pos_y += self.speed * math.sin(angle)
        dir_end_x = self.pos[0] + self.speed * math.cos(angle)
        dir_end_y = self.pos[1] + self.speed * math.sin(angle)

        dir_end = np.array([dir_end_x, dir_end_y], dtype=np.float32)
        direction = dir_end - self.pos
        direction /= np.linalg.norm(direction)

        draw_dir_end = (int(dir_end_x), int(dir_end_y))

        self.angle = self.angle % 360

        self.pos = self.pos + direction * self.speed
        # pos = (self.pos[0], self.pos[1])
        self.center = (int(self.pos[0]), int(self.pos[1]))
        self.rect = pg.draw.circle(screen, RED, (self.center), self.radius)
    
        

In [9]:
angle = math.radians(100)
x=300 + 30 * math.cos(angle)
y=300 + 30 * math.sin(angle)
print(x, y)

294.7905546699921 329.5442325903662


In [10]:
# pg.init()
# pg.display.init()

# window = pg.display.set_mode((WIDTH, HEIGHT))
# clock = pg.time.Clock()
# screen = pg.Surface((WIDTH, HEIGHT))

So every shape inside the pygame is a rect and all the functions of the rect can be used for collide or other stuff for the debug

In [292]:
# just need to take the object inside a variable and use it to need

rect1 = pg.draw.rect(screen, RED, (310, 300, 50, 50))
rect2 = pg.draw.circle(screen, RED, (300, 300), 15)


rect2.colliderect(rect1)

True

In [11]:
pg.init()
pg.display.init()

window = pg.display.set_mode((WIDTH, HEIGHT))
clock = pg.time.Clock()
screen = pg.Surface((WIDTH, HEIGHT))
screen.fill(WHITE)

last_shot_time = 0
fire_rate = 1000
turret = Turret(screen, WIDTH, HEIGHT)
# turret.angle = 45
bullets = []
running = True
total_reward = 0 
while running:
    screen.fill(WHITE)
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
    current_time = pg.time.get_ticks()

    # keys = pg.key.get_pressed()

    # if keys[pg.K_SPACE]:
    if current_time - last_shot_time >= fire_rate:
        turret.shoot()

        last_shot_time = current_time
    else:
        turret.angle += 10

    for bullet in turret.get_bullets():
        bullet.move()
    # bullets = [bullet for bullet in bullets if screen.get_rect().colliderect(bullet.rect)]
    #     # if not screen.get_rect().colliderect(bullet.rect):
    #     #     bullets.remove(bullet)
    #     # else:
    #     #     pg.draw.rect(screen, RED, (bullet.rect.x, bullet.rect.y, bullet.rect.height, bullet.rect.width) )
    

    # pg.draw.rect(screen, GREEN, (turret.rect.x, turret.rect.y, turret.rect.width, turret.rect.height))
    # pg.draw.circle(screen, GREEN, (turret.center), turret.radius)
    
    pg.draw.line(screen, GREEN, turret.center, turret.rotate_turret(), 4)

    wall = pg.draw.rect(screen, (0, 0, 255), (500, 100, 40, 400))
    
    for bullet in turret.get_bullets():
        # pg.draw.circle(screen, RED, (bullet.center), bullet.radius)

        if bullet.pos[0] + 30 > WIDTH or bullet.pos[1] + 30 > HEIGHT:
            turret.destroy_bullet(bullet)

        if bullet.pos[0] - 30 < 0 or bullet.pos[1] - 30 < 0:
            turret.destroy_bullet(bullet)

        if bullet.rect.colliderect(wall):
            turret.destroy_bullet(bullet)
            total_reward += 1
            print(total_reward)

    window.blit(screen, screen.get_rect())
    pg.event.pump()
    pg.display.update()
    clock.tick(60)

pg.display.quit()
pg.quit()

1


In [345]:
pg.display.quit()
pg.quit()